# IE 306.02 Assignment 1
In this assignment we have implemented a call center simulation. Clients place a call to front desk operator who then further forwards them to an expert operator. Clients may renage after front desk operator while waiting for expert, expert takes a break sometimes and both queues are FCFS. Exact details of both queues can be found in assignment description.

We have designed the system as two M/M/1 queues put one after another. In the first queue, server is front desk opetor who we modeled as a resource in the SimPy environment. Similarly, expert is the server in the second queue who is also modeled as a resource in SimPy. 

Below, we first describe our code and later interpret the statistics and results we have gathered from the simulation.

First we import the basic libraries.

In [1]:
import simpy
import random
import numpy.random
import numpy as np
from math import log,sqrt
import itertools # for easy list operations
from collections import namedtuple

A little helper function for formatting seconds to desired time format.

In [2]:
def format_time(total_minutes):
    secs = int((total_minutes % 1)*60)
    mins = int(total_minutes % 60)
    hours = int(total_minutes //60)
    return f" {hours:02}:{mins:02}:{secs:02}"
        
# an object to hold expert queue statistics
Expert_Q_Stat = namedtuple('Expert_Q_Stat', ['enter_e_q', 'exit_e_q'])

Below, we first describe our code and then interpret results.

We ollect and report statistics on: 

* Utilization of the front-desk operator

* Utilization of the expert operators, 

* Average Total Waiting Time,

* `Maximum Total Waiting Time` to `Total System Time Ratio`,

* Average number of people waiting to be served by the expert operator.

Global constants that will be used in the code.

# To run the simulation for 1000 customers, make sure that CUSTOMER_NUM equals CALLNUM1

In [3]:
# number of customers simulation will be run for
CALLNUM1=1000
CALLNUM2=5000

CUSTOMER_NUM=CALLNUM2

# interarrival rates are exponentially distributed with mean 14.3 min
INTERARRIVAL_MEAN = 14.3 
# service time of first operator is LogNormal distributed with mean 7.2 and sd 2.7
m = 7.2
v = 2.7

FIRST_SERVICE_MEAN = log((m**2)/sqrt(v+m**2))
FIRST_SERVICE_STD = sqrt(log(v/(m**2)+1))


# customers leave queue if not serviced in exponentially distributed time with mean 60 min
RENEGE_TIME_MEAN = 60
# service time of expert is exponentially distributed with mean 10.2
EXPERT_SERVICE_MEAN = 10.2
# number of breaks taken by expert is Poisson distributed with mean 8 breaks
BREAKS_MEAN = 8
BREAK_DUR = 3

LAST_EXIT = 0

LAST_CUSTOMER = 1

RANDOM_SEED = 978
random.seed(RANDOM_SEED)

Here, we define the necessary set of arrays for bookkeeping

In [4]:
# service times for front-desk operator
first_service_times = [] 
# service times for expert 
second_service_times = []
# time spent by a customer while it waits for the operator (Queue waiting time Wq)
first_queue_w_times = [] 
second_queue_w_times = [] 


expert_queue_log = []

# counter for breaks taken by the expert
break_count = 0



A said above, we modeled the front desk and expert operators as resources, calls(called customer in code) and breaks that expert takes are modeled as processes. Breaks taken by the expert makes the expert resource unavailable for 3 mins, so it made sense to model breaks as processes that try to get expert resource.

Class definition for breaks. Breaks are generated for expert operator. When they happen, they take 3 minutes long and expert is not available at that time. 

In [5]:
class Break:
    def __init__(self, env, expert):
        self.env = env
        self.expert = expert
        # it only action is happen, which can be thought as just being initiated
        self.action = env.process(self.happen())
    
    
    def happen(self):
        print(f"Break is on the way \t\t\t\t at {format_time(self.env.now)}")
        with self.expert.request() as req:
            yield req
            print(f"Break started \t\t\t\t\t at {format_time(self.env.now)}")
            yield self.env.timeout(BREAK_DUR)
            print(f"Break ended \t\t\t\t\t at {format_time(self.env.now)}")

* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the resource is available, the customer is activated and it then initiates first the ask_question_operator process. The duration of a question-answer session is determined randomly according to a LogNormal distribution.

* After this, they are forwarded to expert operator. While waiting for the expert, they may renege after some time taken from an exponential distribution if they aren't addressed by the expert.

* Service time of the expert is taken from another exponential distribution.


In [6]:
class Customer(object):
    def __init__(self, _id, env, operator, expert):
        self.env = env
        self.id = _id
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    
    def call(self):
        print(f"Cust {self.id:4} entered the \t\t FRONT Queue. \t at {format_time(self.arrival_t)}")
 
        with operator.request() as op_req :
            yield op_req
            self.wait_front = self.env.now - self.arrival_t
            print(f"Cust {self.id:4} started talking with \t FRONT Desk \t at {format_time(self.env.now)}")
            if self.wait_front:
                    print(f"Cust {self.id:4} waited in \t\t FRONT Queue \t for{format_time(self.wait_front)}")
            first_queue_w_times.append(self.wait_front)
            yield self.env.process(self.ask_question_operator())
            self.finish_operator=self.env.now
            print(f"Cust {self.id:4} entered the \t\t EXPERT Queue. \t at {format_time(self.finish_operator)}")
        

        with expert.request() as exp_req: 
                enter_e_q = self.env.now
                # customers may renege in the expert queue
                self.patience=random.expovariate(1/RENEGE_TIME_MEAN)
                results = yield exp_req | env.timeout(self.patience)
                self.wait_exp=self.env.now-self.finish_operator
                second_queue_w_times.append(self.wait_exp) 
                exit_e_q = self.env.now
                global expert_queue_log
                expert_queue_log.append(Expert_Q_Stat(enter_e_q, exit_e_q))

                if exp_req not in results:
                    #Leave queue, reneged
                    print(f"Cust {self.id:4} exited \t\t RENEGE, \t at {format_time(self.env.now)}")
                    print(f"-- after waiting {format_time(self.wait_exp)}")

                else:
                # We got to the expert
                    print(f"Cust {self.id:4} started talking with \t EXPERT \t at {format_time(self.env.now)}")
                    if self.wait_exp:
                        print(f"Cust {self.id:4} waited in \t\t EXPERT Queue \t for{format_time(self.wait_exp)}")
                    yield self.env.process(self.ask_question_expert())
                    self.finish_expert=self.env.now
                    print(f"Cust {self.id:4} exited \t\t NORMALLY \t at {format_time(self.finish_expert)}")
        
        global LAST_EXIT
        LAST_EXIT = self.env.now
            
    def ask_question_operator(self):
        service_opr = random.lognormvariate(FIRST_SERVICE_MEAN,FIRST_SERVICE_STD)
        first_service_times.append(service_opr)
        yield self.env.timeout(service_opr)


    def ask_question_expert(self):
        service_exp = random.expovariate(1/EXPERT_SERVICE_MEAN)
        second_service_times.append(service_exp)
        yield self.env.timeout(service_exp)

Generator functions for customers and breaks. 

* Interrarrival times for both are taken from different exponential distributions. 

* Customer generator runs as many times as the number of total customers we want in our simulation, break generator runs through the whole shift.


In [7]:
def customer_generator(env, operator,expert,callnum):
    for i in range(callnum):
        global LAST_CUSTOMER
        LAST_CUSTOMER = LAST_CUSTOMER + 1
 
        time_between_next_customer = random.expovariate(1/INTERARRIVAL_MEAN)
        customer = Customer(i+1, env, operator,expert)

        yield env.timeout(time_between_next_customer)
        
def break_generator(env, expert):
    while LAST_CUSTOMER<CUSTOMER_NUM:
        time_to_break = random.expovariate(1/60)
 
        yield env.timeout(time_to_break)
        Break(env, expert)
        global break_count
        break_count += 1

Here, we define our processes and resources and start running the simulation environment!

In [8]:

env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
expert=simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator,expert,CUSTOMER_NUM))
env.process(break_generator(env, expert))
end_simulation = simpy.Event(env)
env.run() 

Cust    1 entered the 		 FRONT Queue. 	 at  00:00:00
Cust    1 started talking with 	 FRONT Desk 	 at  00:00:00
Cust    1 entered the 		 EXPERT Queue. 	 at  00:07:10
Cust    1 started talking with 	 EXPERT 	 at  00:07:10
Break is on the way 				 at  00:10:06
Cust    1 exited 		 NORMALLY 	 at  00:25:05
Break started 					 at  00:25:05
Break ended 					 at  00:28:05
Cust    2 entered the 		 FRONT Queue. 	 at  01:03:11
Cust    2 started talking with 	 FRONT Desk 	 at  01:03:11
Break is on the way 				 at  01:06:28
Break started 					 at  01:06:28
Cust    2 entered the 		 EXPERT Queue. 	 at  01:09:16
Break ended 					 at  01:09:28
Cust    2 started talking with 	 EXPERT 	 at  01:09:28
Cust    2 waited in 		 EXPERT Queue 	 for 00:00:12
Cust    3 entered the 		 FRONT Queue. 	 at  01:12:19
Cust    3 started talking with 	 FRONT Desk 	 at  01:12:19
Cust    3 entered the 		 EXPERT Queue. 	 at  01:18:57
Cust    4 entered the 		 FRONT Queue. 	 at  01:32:21
Cust    4 started talking with 	 FRONT D

Cust  294 started talking with 	 FRONT Desk 	 at  73:21:30
Cust  295 entered the 		 FRONT Queue. 	 at  73:23:53
Cust  296 entered the 		 FRONT Queue. 	 at  73:27:05
Cust  294 entered the 		 EXPERT Queue. 	 at  73:28:48
Cust  295 started talking with 	 FRONT Desk 	 at  73:28:48
Cust  295 waited in 		 FRONT Queue 	 for 00:04:55
Cust  295 entered the 		 EXPERT Queue. 	 at  73:36:03
Cust  296 started talking with 	 FRONT Desk 	 at  73:36:03
Cust  296 waited in 		 FRONT Queue 	 for 00:08:58
Cust  293 exited 		 NORMALLY 	 at  73:36:47
Cust  294 started talking with 	 EXPERT 	 at  73:36:47
Cust  294 waited in 		 EXPERT Queue 	 for 00:07:58
Cust  297 entered the 		 FRONT Queue. 	 at  73:39:50
Cust  296 entered the 		 EXPERT Queue. 	 at  73:40:39
Cust  297 started talking with 	 FRONT Desk 	 at  73:40:39
Cust  297 waited in 		 FRONT Queue 	 for 00:00:48
Cust  298 entered the 		 FRONT Queue. 	 at  73:40:39
Cust  297 entered the 		 EXPERT Queue. 	 at  73:50:58
Cust  298 started talking with 	 FRO

Cust  654 started talking with 	 EXPERT 	 at  165:17:46
Cust  654 waited in 		 EXPERT Queue 	 for 00:22:33
Cust  654 exited 		 NORMALLY 	 at  165:17:56
Cust  655 entered the 		 FRONT Queue. 	 at  165:32:50
Cust  655 started talking with 	 FRONT Desk 	 at  165:32:50
Cust  655 entered the 		 EXPERT Queue. 	 at  165:38:33
Cust  655 started talking with 	 EXPERT 	 at  165:38:33
Cust  656 entered the 		 FRONT Queue. 	 at  165:51:56
Cust  656 started talking with 	 FRONT Desk 	 at  165:51:56
Break is on the way 				 at  165:55:22
Cust  656 entered the 		 EXPERT Queue. 	 at  165:58:37
Break is on the way 				 at  165:59:00
Cust  657 entered the 		 FRONT Queue. 	 at  166:12:03
Cust  657 started talking with 	 FRONT Desk 	 at  166:12:03
Cust  657 entered the 		 EXPERT Queue. 	 at  166:18:27
Cust  658 entered the 		 FRONT Queue. 	 at  166:27:04
Cust  658 started talking with 	 FRONT Desk 	 at  166:27:04
Cust  659 entered the 		 FRONT Queue. 	 at  166:27:09
Break is on the way 				 at  166:30:13


Cust  960 waited in 		 FRONT Queue 	 for 00:00:20
Cust  959 exited 		 RENEGE, 	 at  234:54:14
-- after waiting  00:05:20
Cust  960 entered the 		 EXPERT Queue. 	 at  234:55:14
Break is on the way 				 at  235:28:32
Cust  961 entered the 		 FRONT Queue. 	 at  235:32:45
Cust  961 started talking with 	 FRONT Desk 	 at  235:32:45
Break is on the way 				 at  235:35:14
Cust  961 entered the 		 EXPERT Queue. 	 at  235:37:39
Cust  962 entered the 		 FRONT Queue. 	 at  235:39:36
Cust  962 started talking with 	 FRONT Desk 	 at  235:39:36
Cust  963 entered the 		 FRONT Queue. 	 at  235:40:56
Cust  958 exited 		 NORMALLY 	 at  235:42:54
Cust  960 started talking with 	 EXPERT 	 at  235:42:54
Cust  960 waited in 		 EXPERT Queue 	 for 00:47:40
Cust  962 entered the 		 EXPERT Queue. 	 at  235:46:37
Cust  963 started talking with 	 FRONT Desk 	 at  235:46:37
Cust  963 waited in 		 FRONT Queue 	 for 00:05:41
Cust  961 exited 		 RENEGE, 	 at  235:47:46
-- after waiting  00:10:07
Cust  960 exited 		 N

Lastly, we print our statistics we gathered from the simulation

In [9]:
util_front = sum(first_service_times) / LAST_EXIT
util_exp = sum(second_service_times) / LAST_EXIT

total_waits = [f_w + e_w for f_w, e_w in itertools.zip_longest(
    first_queue_w_times, 
    second_queue_w_times, 
    fillvalue=0
)]

total_services = [f_s + e_s for f_s, e_s in itertools.zip_longest(
    first_service_times,
    second_service_times,
    fillvalue=0
)]

total_system = [w + s for w, s in itertools.zip_longest(
    total_waits,
    total_services,
    fillvalue=0
)]

# this makes sure everything went smoothly 
assert(len(total_waits) == len(total_system))

max_ratio = max((w/sys if sys else 0 for w, sys in zip(total_waits, total_system)))

# a helper function to calculate avg length of expert queue
def log_to_avg_len(log):
   ins = sorted((entry.enter_e_q for entry in log))
   outs = sorted((entry.exit_e_q for entry in log))
   significant_points = np.array(sorted(ins + outs))
   in_mask = np.array([i in ins for i in significant_points])
   out_mask = np.array([o in outs for o in significant_points])
   queue_len = np.zeros(len(significant_points))

   
   for i, is_in in enumerate(in_mask):
      if is_in:
         queue_len[i:] += 1

   for i, is_in in enumerate(out_mask):
      if is_in:
         queue_len[i:] -= 1

   timedeltas = np.copy(significant_points)
   for i, elem in enumerate(timedeltas):
      timedeltas[i+1:] -= elem

   return np.sum( timedeltas*queue_len ) / outs[-1]

stats = {
    "break_count": break_count,
    "util_front": util_front,
    "util_exp": util_exp,
    "avg_total_wait": sum(total_waits) / len(total_waits),
    "max_wait_2_sys_ratio": max_ratio,
    "avg_len_of_exp_q": log_to_avg_len(expert_queue_log),
}

print("Number of breaks taken: %d" % (stats["break_count"]))
print("Utilization of front desk operator: %%%f" % (stats["util_front"]))
print("Utilization of expert operator: %%%f" % (stats["util_exp"]))
print("Average total waiting time: %f" % (stats["avg_total_wait"]))
print("Maximum of total waiting time to system time ratios: %%%f" % (stats["max_wait_2_sys_ratio"]))
print("Average length of expert waiting queue: %f" % (stats["avg_len_of_exp_q"]))


Number of breaks taken: 254
Utilization of front desk operator: %0.487187
Utilization of expert operator: %0.597796
Average total waiting time: 11.670236
Maximum of total waiting time to system time ratios: %0.894770
Average length of expert waiting queue: 0.687378


In [10]:
print (len(first_queue_w_times))

1000


In [11]:
print (len(second_queue_w_times))

1000


In [12]:
print(format_time(env.now))
print(format_time(LAST_EXIT))

 246:54:02
 244:56:58


In [13]:
print(second_service_times)

[17.912965238881462, 37.241018815004686, 0.2173634977609906, 8.06217892182097, 20.260319394182773, 1.014941251016786, 4.369103959285335, 12.194022905029676, 3.2567733625916957, 6.780544858214905, 2.2241755369806118, 14.340097210490132, 5.086755499464572, 4.837472377295497, 12.631066746526537, 11.459785139751999, 0.917153118236034, 0.06096492339018591, 3.5013982643669803, 13.218615304924223, 5.9972468648859625, 19.498913263938938, 2.679171928072061, 2.307605602067655, 1.1284683382654173, 1.2975966968403063, 7.197120829228969, 13.13863677655889, 9.386583807876704, 13.657847533512614, 1.132432155271038, 7.643842905042442, 12.315170257475486, 0.21854537740737373, 54.01530086142748, 18.076807789795055, 1.8330956639122595, 4.209351745953881, 8.982120831477575, 6.717074444480771, 14.914029989129224, 3.234605548606124, 6.51229136390839, 10.966795347856495, 11.28678631329193, 4.586318824108399, 3.0250618460508427, 0.11866537910449329, 0.8734908635146335, 9.18767230834964, 12.166263041122667, 6.

In [14]:
print(sum(first_service_times))

7160.169759988991
